In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torchvision.datasets import ImageFolder
from torch.utils.data import random_split, DataLoader

# Set up paths to your data
num_classes = 18  # Set this to the number of classes you have
batch_size = 32
num_epochs = 100


# data_dir = '/kaggle/input/pattern-recognition/pattern-recognition'
data_dir = '/kaggle/input/fabric-romo/romo'

transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

dataset = ImageFolder(data_dir+'/train', transform=transform)
img, label = dataset[0]
print(img.size(), label)
print(dataset.classes)


print(len(dataset))
total_size = len(dataset)
val_size = int(0.3 * total_size)
train_size = len(dataset) - val_size

train_ds, val_ds = random_split(dataset, [train_size, val_size])
train_loader = DataLoader(train_ds, batch_size, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_ds, batch_size, num_workers=4, pin_memory=True)


# Load the pre-trained EfficientNet model
model = models.efficientnet_b0(pretrained=True)

# Modify the classifier layer to match the number of classes
model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)

# Use GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Training loop
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}')

# Validation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Validation Accuracy: {100 * correct / total}%')

# Fine-tuning the model (optional)
for param in model.features.parameters():
    param.requires_grad = False

optimizer = optim.Adam(model.parameters(), lr=1e-5)

# Training loop for fine-tuning
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    print(f'Fine-tuning Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}')

# Validation loop after fine-tuning
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Validation Accuracy after Fine-tuning: {100 * correct / total}%')

torch.save(model.state_dict(), "/kaggle/working/model.pth")